In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import json
from ast import literal_eval # 걍 eval쓰면 스트링을 배열로 만들 수 있다.

/home/leadawon5/gitfiles/venvs/huggvenv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import numpy as np

from transformers import AutoTokenizer, DPRQuestionEncoder, DPRContextEncoder
from typing import List

from pprint import pprint

class BiEncoderRetriever:
    def __init__(self) -> None:
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.tokenizer = AutoTokenizer.from_pretrained("sivasankalpp/dpr-multidoc2dial-structure-question-encoder")
        self.question_encoder = DPRQuestionEncoder.from_pretrained("sivasankalpp/dpr-multidoc2dial-structure-question-encoder").to(self.device)
        self.ctxt_encoder = DPRContextEncoder.from_pretrained("sivasankalpp/dpr-multidoc2dial-structure-ctx-encoder").to(self.device)

    def encode_summaries(self, summaries: List[str]):
        input_dict = self.tokenizer(summaries, padding='max_length', max_length=128, truncation=True, return_tensors="pt").to(self.device)
        del input_dict["token_type_ids"]
        return self.ctxt_encoder(**input_dict)['pooler_output']

    def encode_question(self, question: str):
        input_dict = self.tokenizer(question, padding='max_length', max_length=32, truncation=True, return_tensors="pt").to(self.device)
        del input_dict["token_type_ids"]
        return self.question_encoder(**input_dict)['pooler_output']

    def retrieve_top_summaries(self, question: str, summaries: List[str], encoded_summaries: np.ndarray = None, topk: int = 5):
        encoded_question = self.encode_question(question)
        if encoded_summaries is None:
            encoded_summaries = self.encode_summaries(summaries)
        else:
            encoded_summaries = torch.from_numpy(encoded_summaries).to(self.device)

        scores = torch.mm(encoded_question, encoded_summaries.T)
        # print(encoded_question.shape)
        # print(encoded_summaries.T.shape)
        if topk >= len(summaries):
            return summaries
        top_k = torch.topk(scores, topk).indices.squeeze()
        
        #print("all scores : ")
        #for i,v in enumerate(scores[0]):
            #print(int(v.item()),end=" ")
        #print()
        
        return [summaries[i] for i in top_k]

In [4]:
rt = BiEncoderRetriever()

In [7]:
with open("./bigdata/persona_output.jsonl","w") as fout:
    print("start!")

with open("./bigdata/extracted_persona_session3_v1.jsonl", "r") as dialogs:
    for num,dialog in enumerate(dialogs):
#         if num == 50:
#             break;
#         else:
        print(f"now {num} dialog")
        dic = json.loads(dialog)
        dialog_list = dic["current"]
        old_persona_list = dic["session1_persona"] + dic["session2_persona"]
        new_persona_list_1 = []
        new_persona_list_2 = []
        for i,v in enumerate(old_persona_list):
            if(v.split()[1]=="1"):
                new_persona_list_1.append("Speaker "+" ".join(v.split()[2:]))
            elif(v.split()[1]=="2"):
                new_persona_list_2.append("Speaker "+" ".join(v.split()[2:]))
            else:
                print(v.split()[1])
                new_persona_list_2.append("Speaker "+" ".join(v.split()[2:]))
        
        
        with open("./bigdata/persona_output.jsonl","a") as fout:
            resultlist = []
            for i,v in enumerate(dialog_list):
                
                question = " ".join(v.split()[2:])
                if(v.split()[1]=="1:"):
                    personalist = new_persona_list_2
                elif(v.split()[1]=="2:"):
                    personalist = new_persona_list_1
                else:
                    print(v.split()[1])
                    personalist = new_persona_list_1
                resultlist.append(rt.retrieve_top_summaries(question, personalist, None, topk=3))
            epl_list = []
            for d,r in zip(dialog_list,resultlist):
                epl_list.append({"utterance":d, "3persona":r})
            #newdic = {"current" : dialog_list, "persona_list": persona_list, "extracted_persona_list":epl_list}
            newdic = {"persona_list_1": new_persona_list_1,"persona_list_2": new_persona_list_2, "extracted_persona_list":epl_list}
            fout.write(json.dumps(newdic, ensure_ascii=False) + "\n")

start!
now 0 dialog
now 1 dialog
now 2 dialog
now 3 dialog
now 4 dialog
now 5 dialog
now 6 dialog
now 7 dialog
now 8 dialog
now 9 dialog
now 10 dialog
now 11 dialog
now 12 dialog
now 13 dialog
now 14 dialog
now 15 dialog
now 16 dialog
now 17 dialog
now 18 dialog
now 19 dialog
now 20 dialog
now 21 dialog
now 22 dialog
now 23 dialog
now 24 dialog
now 25 dialog
now 26 dialog
now 27 dialog
now 28 dialog
now 29 dialog
now 30 dialog
now 31 dialog
now 32 dialog
now 33 dialog
now 34 dialog
now 35 dialog
now 36 dialog
1:
now 37 dialog
now 38 dialog
now 39 dialog
now 40 dialog
now 41 dialog
now 42 dialog
now 43 dialog
now 44 dialog
now 45 dialog
now 46 dialog
now 47 dialog
now 48 dialog
now 49 dialog
now 50 dialog
now 51 dialog
now 52 dialog
now 53 dialog
now 54 dialog
now 55 dialog
now 56 dialog
now 57 dialog
now 58 dialog
now 59 dialog
now 60 dialog
now 61 dialog
now 62 dialog
now 63 dialog
now 64 dialog
now 65 dialog
now 66 dialog
now 67 dialog
now 68 dialog
now 69 dialog
now 70 dialog
now 71

In [8]:
question = "What is Sarah's favorite animal?"
personalist = ['Sarah is 24 years old.', 
               'Sarah currently lives in Canada.', 
               "Sarah is a swim coach at Sarah's local pool.", 
               'Sarah is studying to be a computer programmer.', 
               'Sarah is also a graduate student.', 
               'Sarah is now looking for a new job.', 
               "Sarah's mother is very traditional while Sarah prefers to be more free spirited.", 
               "Sarah's family and Sarah are from India.", 
               "Sarah's favorite music genre is death metal.", 
               'Sarah is a famous twitch streamer.', 
               'Sarah likes watching war documentaries.', 
               "Sarah's favorite food is mexican food.",
               "What is Sarah's favorite animal?"
              ]
pprint(rt.retrieve_top_summaries( # here is using encodings.?!@
                question, personalist, None, topk=3
            ))

["What is Sarah's favorite animal?",
 "Sarah's favorite food is mexican food.",
 "Sarah's mother is very traditional while Sarah prefers to be more free "
 'spirited.']
